### extracting features from intermediate blocks of a CNN (using keras):

In [1]:
import tensorflow as tf
tf.random.set_seed(1)
import random
random.seed(1)

import tensorflow as tf
import numpy as np
import pandas as pd

from utils import DataGenerator
import keras.backend as K
from pathlib import Path


import gc

2024-10-28 15:05:18.699325: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-28 15:05:18.744960: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [6]:
def get_layer_output(model, layer_name):
    '''
    get features from an intermediate layer of a model using a data generator
    
    model: saved model you want to extract features from
    layer_name: name of layer to take feature activations from
    '''
    
    #load file with test set images
    fn_test = working_dir + "test.csv" 
    test = pd.read_csv(fn_test, index_col = 'filepath')

    test_fpaths=test.index.to_numpy()

    #we extract activations in "chunks" of 50 images to not overload memory
    chunk_size = 50
    for i in range(0, len(test_fpaths), chunk_size):
        chunk = test_fpaths[i:i+chunk_size]
        test_generator=DataGenerator(chunk, params['batch_size'],
                                  (params['imagex'], params['imagey'], params['imagez']), False, fn_test)

        #create "intermediate model" that only includes layers up to the layer of interest
        layer_output = model.get_layer(layer_name).output
        flatten_layer = tf.keras.layers.Flatten()(layer_output)
        intermediate_model = tf.keras.models.Model(inputs=model.input, outputs=flatten_layer)
        intermediate_model.summary()
        
        #get output from images through intermediate model
        pred = intermediate_model.predict(test_generator)

        #flatten along all dimensions other than first
        pred_flat = pred.reshape(pred.shape[0], -1)
        pred_flat = pred_flat.astype(np.float32)

        if i == 0: #if this is the first "chunk", create a new np array 
            sname = save_dir + save_name + '_' + layer_name
            np.save(sname, pred_flat)

        else: #if this is not the first "chunk", add new features to existing array 
            features_old = np.load(sname + '.npy')
            features = np.concatenate((features_old, pred_flat))
            sname = save_dir + save_name + '_' + layer_name
            np.save(sname, features)

            del features_old
            del features

        #clear up memory
        K.clear_session()
        del pred
        del pred_flat
        del intermediate_model
        gc.collect()

    return

In [3]:
exp_name = 'morph_bias'
working_dir = '/home/emma/Documents/SBB/' + exp_name + '/'
model_name = 'best_model_' + exp_name + '.h5'
save_dir = working_dir + 'split/'
save_name = exp_name + '_features'
Path(save_dir).mkdir(parents=True, exist_ok=True)
params = {'batch_size': 1,
        'imagex': 173,
        'imagey': 211,
        'imagez': 155,
        }

In [4]:
# import model
model = tf.keras.models.load_model(working_dir + model_name)

2024-10-28 15:05:23.161677: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-10-28 15:05:23.181987: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-10-28 15:05:23.182192: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-10-28 15:05:23.182583: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFl

In [5]:
layers = ['mp3', 'mp4', 'mp5', 'flatten1']

#get features from model
for layer_name in layers:
    get_layer_output(model, layer_name)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 InputA (InputLayer)         [(None, 173, 211, 155, 1  0         
                             )]                                  
                                                                 
 conv1 (Conv3D)              (None, 173, 211, 155, 32  896       
                             )                                   
                                                                 
 bn1 (BatchNormalization)    (None, 173, 211, 155, 32  128       
                             )                                   
                                                                 
 act1 (Activation)           (None, 173, 211, 155, 32  0         
                             )                                   
                                                                 
 mp1 (MaxPooling3D)          (None, 87, 106, 78, 32)   0     

2024-10-28 15:05:26.461459: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8401
2024-10-28 15:05:27.275816: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-10-28 15:05:27.276144: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-10-28 15:05:27.276161: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2024-10-28 15:05:27.276462: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-10-28 15:05:27.276493: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


50/50 [==============================] - 5s 63ms/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 InputA (InputLayer)         [(None, 173, 211, 155, 1  0         
                             )]                                  
                                                                 
 conv1 (Conv3D)              (None, 173, 211, 155, 32  896       
                             )                                   
                                                                 
 bn1 (BatchNormalization)    (None, 173, 211, 155, 32  128       
                             )                                   
                                                                 
 act1 (Activation)           (None, 173, 211, 155, 32  0         
                             )                                   
                                                                 
 mp1 (M

 conv1 (Conv3D)              (None, 173, 211, 155, 32  896       
                             )                                   
                                                                 
 bn1 (BatchNormalization)    (None, 173, 211, 155, 32  128       
                             )                                   
                                                                 
 act1 (Activation)           (None, 173, 211, 155, 32  0         
                             )                                   
                                                                 
 mp1 (MaxPooling3D)          (None, 87, 106, 78, 32)   0         
                                                                 
 conv2 (Conv3D)              (None, 87, 106, 78, 64)   55360     
                                                                 
 bn2 (BatchNormalization)    (None, 87, 106, 78, 64)   256       
                                                                 
 act2 (Act

                             )                                   
                                                                 
 mp1 (MaxPooling3D)          (None, 87, 106, 78, 32)   0         
                                                                 
 conv2 (Conv3D)              (None, 87, 106, 78, 64)   55360     
                                                                 
 bn2 (BatchNormalization)    (None, 87, 106, 78, 64)   256       
                                                                 
 act2 (Activation)           (None, 87, 106, 78, 64)   0         
                                                                 
 mp2 (MaxPooling3D)          (None, 44, 53, 39, 64)    0         
                                                                 
 conv3 (Conv3D)              (None, 44, 53, 39, 128)   221312    
                                                                 
 bn3 (BatchNormalization)    (None, 44, 53, 39, 128)   512       
          

...

                                                                 
 avgpool1 (AveragePooling3D)  (None, 3, 4, 3, 512)     0         
                                                                 
 dropout1 (Dropout)          (None, 3, 4, 3, 512)      0         
                                                                 
 flatten1 (Flatten)          (None, 18432)             0         
                                                                 
 flatten (Flatten)           (None, 18432)             0         
                                                                 
Total params: 4,705,984
Trainable params: 4,704,000
Non-trainable params: 1,984
_________________________________________________________________
50/50 [==============================] - 3s 66ms/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 InputA (InputLayer)         [(None, 173, 211, 155, 1  0   

                                                                 
 dropout1 (Dropout)          (None, 3, 4, 3, 512)      0         
                                                                 
 flatten1 (Flatten)          (None, 18432)             0         
                                                                 
 flatten (Flatten)           (None, 18432)             0         
                                                                 
Total params: 4,705,984
Trainable params: 4,704,000
Non-trainable params: 1,984
_________________________________________________________________
50/50 [==============================] - 3s 66ms/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 InputA (InputLayer)         [(None, 173, 211, 155, 1  0         
                             )]                                  
                                                           

                                                                 
 flatten1 (Flatten)          (None, 18432)             0         
                                                                 
 flatten (Flatten)           (None, 18432)             0         
                                                                 
Total params: 4,705,984
Trainable params: 4,704,000
Non-trainable params: 1,984
_________________________________________________________________
50/50 [==============================] - 3s 66ms/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 InputA (InputLayer)         [(None, 173, 211, 155, 1  0         
                             )]                                  
                                                                 
 conv1 (Conv3D)              (None, 173, 211, 155, 32  896       
                             )                             

                                                                 
 flatten (Flatten)           (None, 18432)             0         
                                                                 
Total params: 4,705,984
Trainable params: 4,704,000
Non-trainable params: 1,984
_________________________________________________________________
50/50 [==============================] - 3s 65ms/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 InputA (InputLayer)         [(None, 173, 211, 155, 1  0         
                             )]                                  
                                                                 
 conv1 (Conv3D)              (None, 173, 211, 155, 32  896       
                             )                                   
                                                                 
 bn1 (BatchNormalization)    (None, 173, 211, 155, 32  128 

                                                                 
Total params: 4,705,984
Trainable params: 4,704,000
Non-trainable params: 1,984
_________________________________________________________________
50/50 [==============================] - 3s 65ms/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 InputA (InputLayer)         [(None, 173, 211, 155, 1  0         
                             )]                                  
                                                                 
 conv1 (Conv3D)              (None, 173, 211, 155, 32  896       
                             )                                   
                                                                 
 bn1 (BatchNormalization)    (None, 173, 211, 155, 32  128       
                             )                                   
                                                           